In [6]:
import os
import numpy as np
import pandas as pd
import networkx as nx
from dask.distributed import Client, progress, wait
from IPython.display import display, HTML
from sklearn.metrics import roc_auc_score
import project_path
from src.util.generate_connected_graph import generate_connected_graph
from src.util.generate_lr_data import generate_low_rank_data
from src.util.generate_anomaly import generate_spatio_temporal_anomaly
from src.algos.lr_stss import lr_stss

In [7]:
client = Client(threads_per_worker=1, n_workers=125)
client

/mnt/home/indibimu/miniconda3/lib/python3.9/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 46685 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:46685/status,
Dashboard: http://127.0.0.1:46685/status,Workers: 125
Total threads: 125,Total memory: 80.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:44923,Workers: 125
Dashboard: http://127.0.0.1:46685/status,Total threads: 125
Started: Just now,Total memory: 80.00 GiB
Comm: tcp://127.0.0.1:39140,Total threads: 1
Dashboard: http://127.0.0.1:40816/status,Memory: 655.36 MiB
Nanny: tcp://127.0.0.1:41923,


In [17]:
## Control Variables
gt = 'grid'
graph_param = (8,5)
dtn = 4
rad =1
amp =1
window = 'boxcar'
distribution = 'bernoulli'
local_distribution = 'constant'
dims = (40,24,7,4)
ranks = (8,8,5,4)
NoA = 100
NoG = 1
NoT = 5
time_m = 2
local_m = 1
lda_2 = 10
psi = 20
maxit = 300
## Independent variables
### Hyperparameters
lda_1 = np.logspace(1,-3,5)
lda_l = np.logspace(1,-3,5)
lda_t = np.logspace(1,-3,5)

metadata = {'graph_type': gt,
            'graph_param': graph_param,
            'number_of_anomalies': NoA,
            'anomaly_duration': dtn,
            'anomaly_radius': rad,
            'anomaly_amplitude': amp,
            'window': window,
            'distribution': distribution,
            'local_distribution': local_distribution,
            'number_of_graphs': NoG,
            'number_of_trials': NoT,
            'ranks':ranks,
            'dims':dims,
            'time_mode': time_m,
            'local_mode': local_m,
            'lda_2':lda_2,
            'lda_1':lda_1,
            'lda_l':lda_l,
            'lda_t':lda_t,
            'psi': psi,
            'maxit':maxit,
            'maxit2':maxit2,
            }

# Define experiment function
def run_exp(X, Y, an_m, inputs):
    G = inputs['G']
    A = nx.adjacency_matrix(G)
    Deg = np.diag(np.asarray(np.sum(A,axis=1)).ravel())
    Dsq = np.linalg.inv(np.sqrt(Deg))
    An = Dsq@A@Dsq
    Y = inputs['Y']
    Y = np.ma.masked_array(Y, mask=np.zeros(dims,dtype=bool))
    X = inputs['X']
    an_m = inputs['an_m']
    psi = inputs['psi']
    res = lr_stss(Y, An, time_m,local_m, verbose=0, max_it2=40, max_it=300,
        lda2=inputs['lda_2'], lda1=inputs['lda_1'], lda_t=inputs['lda_t'],
        lda_loc=inputs['lda_l'], psis=[psi,psi,psi,psi])
    
    result = {'graph_type': gt,
              'auc': roc_auc_score(an_m.ravel(),np.abs(res['S']).ravel()),
              'rec_err': np.linalg.norm(res['X']-X)/np.linalg.norm(X),
              'anomaly_cardinality': an_m.sum(),
              'number_of_anomalies': NoA,
              'anomaly_duration': dtn,
              'anomaly_radius': rad,
              'graph_seed': inputs['graph_seed'],
              'anomaly_seed': inputs['anomaly_seed'],
              'anomaly_amplitude': amp,
              'lda_1': inputs['lda_1'],
              'lda_2': inputs['lda_2'],
              'lda_l': inputs['lda_l'],
              'lda_t': inputs['lda_t'],
              'psi_1': inputs['psi'], 'psi_2': inputs['psi'],
              'psi_3': inputs['psi'], 'psi_4': inputs['psi'],
              'maxit': maxit, #'maxit2': maxit2,
              'rho': res['rho'][-1],
              'it': res['it']}
    return result

In [18]:
try:
    del futures
finally:
    pass
futures = []
seed = 46513213
input = {}
for i in range(NoG):
    G,sd = generate_connected_graph(graph_param, gt, seed=seed)
    input['G'] = G
    input['graph_type'] = gt
    input['graph_seed'] = sd
    seed = sd
    for j in range(NoT):
        # Generate low rank normal data
        X = generate_low_rank_data(dims, ranks, seed)
        X = 1*X/np.std(X)
        input['X'] = X
        # Generate anomaly
        an, an_m = generate_spatio_temporal_anomaly(dims, G, NoA, duration=dtn,
                                                    radius=rad, seed=seed,
            time_m=time_m, local_m=local_m, window_type='boxcar', amplitude=amp,
            distribution=distribution, local_dist='constant')
        input['anomaly_seed'] = seed
        input['an_m'] = an_m
        Y = X+an
        # Y = np.ma.masked_array(Y, mask=np.zeros(dims,dtype=bool))
        input['Y'] = Y
        seed +=1
        for k1 in range(len(lda_1)):
            for k2 in range(len(lda_l)):
                for k3 in range(len(lda_t)):
                    input['lda_1'] = lda_1[k1]
                    input['lda_l'] = lda_l[k2]
                    input['lda_t'] = lda_t[k3]
                    input['lda_2'] = lda_2
                    input['psi'] = psi
                    futures.append(client.submit(run_exp, X, Y, an_m, input))
            

Graph is connected.


In [ ]:
progress(futures)

In [ ]:
fname = 'lr_stss_hyperparameter_study_hpcc.hdf5'
key = 'grid'
cwd = os.getcwd()
fname = os.path.join(cwd, 'results', fname)

for i in range(len(futures)):
    df = pd.DataFrame(futures[i].result(), index=[i])
    with pd.HDFStore(fname) as storedata:
        storedata.append(key, df)

with pd.HDFStore(fname) as storedata:
    storedata.get_storer(key).attrs.metadata=metadata

In [5]:
fname = 'lr_stss_hyperparameter_study_hpcc.hdf5'
key = 'grid'
cwd = os.getcwd()
fname = os.path.join(cwd, 'results', fname)
with pd.HDFStore(fname) as storedata:
    df = storedata[key]
    metadata = storedata.get_storer(key).attrs.metadata
# display(HTML(df.to_html()))
# df.info()
print("Metadata:\n",metadata)

Metadata:
 {'graph_type': 'grid', 'graph_param': (8, 5), 'number_of_anomalies': 100, 'anomaly_duration': 4, 'anomaly_radius': 1, 'anomaly_amplitude': 1, 'window': 'boxcar', 'distribution': 'bernoulli', 'local_distribution': 'constant', 'number_of_graphs': 1, 'number_of_trials': 5, 'ranks': (8, 8, 5, 4), 'dims': (40, 24, 7, 4), 'time_mode': 2, 'local_mode': 1, 'lda_2': 10, 'lda_1': array([1.e+01, 1.e+00, 1.e-01, 1.e-02, 1.e-03]), 'lda_l': array([1.e+01, 1.e+00, 1.e-01, 1.e-02, 1.e-03]), 'lda_t': array([1.e+01, 1.e+00, 1.e-01, 1.e-02, 1.e-03]), 'psi': 20, 'maxit': 300, 'maxit2': 40}


In [3]:
from IPython.display import display, HTML
dfm = df.groupby(['lda_1', 'lda_t', 'lda_l'])[['auc', 'rec_err']].mean()
display(HTML(dfm.to_html()))

In [ ]:
dfm.

In [24]:
client.shutdown()

2023-06-26 07:47:04,448 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 233.66 MiB -- Worker memory limit: 327.68 MiB
2023-06-26 07:47:04,523 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 235.30 MiB -- Worker memory limit: 327.68 MiB
2023-06-26 07:47:04,590 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more informati